In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [ ]:
import pandas as pd
import numpy as np

from gensim.models.word2vec import Word2Vec
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
mypath = "/home/kesci/work/drift_data/"
path = "/home/kesci/input/bytedance/"

### 拿出前1E数据和最后1E数据，单独存储

In [ ]:
# 取出前1E数据
train = pd.read_csv("/home/kesci/input/bytedance/train_final.csv",header=None,names=["query_id","query","query_title_id","title","label"],
                    nrows=100000000)
train.to_csv(mypath+"train_first_100million.csv",index=False,header=None)
# 取出最后1E数据
train = pd.read_csv("/home/kesci/input/bytedance/train_final.csv",header=None,names=["query_id","query","query_title_id","title","label"],
                    skiprows = 900000000)
train.to_csv("/home/kesci/work/train_last_100million.csv",index=False)

# 一、word2vec训练

### 训练前1E数据word2vec

In [ ]:
train = pd.read_csv(mypath+"train_first_100million.csv",header=None,names=["query_id","query","query_title_id","title","label"],
                    chunksize=10000000)
test = pd.read_csv(path+"test_final_part1.csv",header=None,names=["query_id","query","query_title_id","title"])

In [ ]:
#训练train
class Word2VecTrainingMaster():
    def __init__(self, work_path=mypath, base_name='drift_word2vec.model', embed_size=300, min_word_count=2,
                 window=5, auto=True, step_save=False):

        self.work_path = work_path
        self.base_name = base_name
        self.word2vec_model = self.make_word2vec_model(embed_size, min_word_count, window)
        self.word2vec_model.save(self.work_path + '/' + self.base_name)
        self.step_save = step_save

        if auto:
            INIT = True
            for chunk in train:
                print(chunk.shape)
                corpus = list(chunk["title"].apply(lambda row : row.split())) + list(chunk["query"].apply(lambda row : row.split()))
                print(corpus[-1])
                self.update_model(corpus, init=INIT)
                INIT = False
                self.word2vec_model.save(self.work_path + '/' + self.base_name)
                print(len(self.word2vec_model.wv.vocab))

    def update_model(self, batch, init):
        if init:
            self.word2vec_model.build_vocab(batch)
        else:
            self.word2vec_model.build_vocab(batch, update=True)
        self.word2vec_model.train(batch, total_examples=self.word2vec_model.corpus_count,
                                  epochs=self.word2vec_model.iter)
        if self.step_save:
            self.word2vec_model.save(self.work_path + '/' + self.base_name)

    def make_word2vec_model(self, embed_size, min_word_count, window):
        return Word2Vec(size=embed_size, min_count=min_word_count, window=window, workers=100, seed=2019, iter=10)

w2v = Word2VecTrainingMaster(embed_size=300)
print(len(w2v.word2vec_model.wv.vocab))

### 测试集1 2000W训练word2vec

In [ ]:
class Word2VecTrainingMaster():
    def __init__(self, work_path=mypath, base_name='drift_word2vec.model', embed_size=300, min_word_count=2,
                 window=5, auto=True, step_save=False):

        self.work_path = work_path
        self.base_name = base_name
        self.word2vec_model = self.make_word2vec_model(embed_size, min_word_count, window)
        #self.word2vec_model.save(self.work_path + '/' + self.base_name)
        self.step_save = step_save
        
        print(len(self.word2vec_model.wv.vocab))
        
        if auto:
            INIT = False
            corpus = list(test["title"].apply(lambda row : row.split())) + list(test["query"].apply(lambda row : row.split()))
            self.update_model(corpus, init=INIT)
            self.word2vec_model.save(self.work_path + '/' + self.base_name)
            print(len(self.word2vec_model.wv.vocab))

    def update_model(self, batch, init):
        if init:
            self.word2vec_model.build_vocab(batch)
        else:
            self.word2vec_model.build_vocab(batch, update=True)
        self.word2vec_model.train(batch, total_examples=self.word2vec_model.corpus_count,
                                  epochs=self.word2vec_model.iter)
        if self.step_save:
            self.word2vec_model.save(self.work_path + '/' + self.base_name)

    def make_word2vec_model(self, embed_size, min_word_count, window):
        #return Word2Vec(size=embed_size, min_count=min_word_count, window=window, workers=4, seed=2019, iter=5)
        return Word2Vec.load("/home/kesci/work/drift_data/drift_word2vec.model")

In [ ]:
model = Word2VecTrainingMaster()

### 最后一亿数据训练word2vec

In [ ]:
train = pd.read_csv("/home/kesci/work/train_last_100million.csv",chunksize=20000000)

In [ ]:
#训练train
class Word2VecTrainingMaster():
    def __init__(self, work_path=mypath, base_name='drift_word2vec_v99.model', embed_size=300, min_word_count=5,
                 window=5, auto=True, step_save=False):

        self.work_path = work_path
        self.base_name = base_name
        self.word2vec_model = self.make_word2vec_model(embed_size, min_word_count, window)
        #self.word2vec_model.save(self.work_path + '/' + self.base_name)
        self.step_save = step_save

        if auto:
            INIT = False
            for chunk in train:
                print(chunk.shape)
                corpus = list(chunk["title"].apply(lambda row : row.split())) + list(chunk["query"].apply(lambda row : row.split()))
                print(corpus[0])
                self.update_model(corpus, init=INIT)
                self.word2vec_model.save(self.work_path + '/' + self.base_name)
                print(len(self.word2vec_model.wv.vocab))

    def update_model(self, batch, init):
        if init:
            self.word2vec_model.build_vocab(batch)
        else:
            self.word2vec_model.build_vocab(batch, update=True)
        self.word2vec_model.train(batch, total_examples=self.word2vec_model.corpus_count,
                                  epochs=self.word2vec_model.iter)
        if self.step_save:
            self.word2vec_model.save(self.work_path + '/' + self.base_name)

    def make_word2vec_model(self, embed_size, min_word_count, window):
        #return Word2Vec(size=embed_size, min_count=min_word_count, window=window, workers=100, seed=2019, iter=10)
        return Word2Vec.load("/home/kesci/work/drift_data/drift_word2vec.model")

In [ ]:
w2v = Word2VecTrainingMaster(embed_size=300)
print(len(w2v.word2vec_model.wv.vocab))

### 测试集2 1E训练word2vec

In [ ]:
test = pd.read_csv("/home/kesci/input/bytedance/bytedance_contest.final_2.csv",names=["query_id","query","query_title_id",
                                        "title"],chunksize=20000000)

In [ ]:
#训练train
class Word2VecTrainingMaster():
    def __init__(self, work_path=mypath, base_name='drift_word2vec_v100.model', embed_size=300, min_word_count=4,
                 window=5, auto=True, step_save=False):

        self.work_path = work_path
        self.base_name = base_name
        self.word2vec_model = self.make_word2vec_model(embed_size, min_word_count, window)
        #self.word2vec_model.save(self.work_path + '/' + self.base_name)
        self.step_save = step_save

        if auto:
            INIT = False
            for chunk in test:
                print(chunk.shape)
                corpus = list(chunk["title"].apply(lambda row : row.strip().split())) + list(chunk["query"].apply(lambda row : row.strip().split()))
                print(corpus[0])
                self.update_model(corpus, init=INIT)
                self.word2vec_model.save(self.work_path + '/' + self.base_name)
                print(len(self.word2vec_model.wv.vocab))

    def update_model(self, batch, init):
        if init:
            self.word2vec_model.build_vocab(batch)
        else:
            self.word2vec_model.build_vocab(batch, update=True)
        self.word2vec_model.train(batch, total_examples=self.word2vec_model.corpus_count,
                                  epochs=self.word2vec_model.iter)
        if self.step_save:
            self.word2vec_model.save(self.work_path + '/' + self.base_name)

    def make_word2vec_model(self, embed_size, min_word_count, window):
        #return Word2Vec(size=embed_size, min_count=min_word_count, window=window, workers=1

In [ ]:
w2v = Word2VecTrainingMaster(embed_size=300)
print(len(w2v.word2vec_model.wv.vocab))

# 二、生成词典映射Tokenizer

In [ ]:
import pandas as pd
import numpy as np
#import fastText 
import keras 

from tqdm import tqdm
from gensim.models.word2vec import Word2Vec
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.callbacks import *
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.externals import joblib

import warnings
warnings.filterwarnings("ignore")

In [ ]:
path = "/home/kesci/input/bytedance/"
mypath = "/home/kesci/work/drift_data/"
# train_path = mypath+"train_first_100million.csv"
test_path = mypath+"test_final_part1.csv"

batch_size = 1024
num_words = 900000
embed_size = 300
query_max_sequence_length = 10
title_max_sequence_length = 22
max_sequence_length = 22

In [ ]:
train = pd.read_csv("/home/kesci/work/train_last_100million.csv")#,nrows=20000000)
print(train.shape)

In [ ]:
train_query = train["query"].drop_duplicates()
train_title = train["title"].drop_duplicates()

In [ ]:
test_stage1 = pd.read_csv("/home/kesci/input/bytedance/test_final_part1.csv",header=None,names=["query_id","query","query_title_id","title"])
test_stage1["title"] = test_stage1["title"].apply(lambda row : row.strip())
print(test_stage1.shape)

In [ ]:
test_query_1 = test_stage1["query"].drop_duplicates()
test_title_1 = test_stage1["title"].drop_duplicates()

In [ ]:
import gc
del train,test_stage1
gc.collect()

In [ ]:
test_stage2 = pd.read_csv("/home/kesci/input/bytedance/bytedance_contest.final_2.csv",header=None,names=["query_id","query","query_title_id","title"])
test_stage2["title"] = test_stage2["title"].apply(lambda row : row.strip())
print(test_stage2.shape)

In [ ]:
test_query_2 = test_stage2["query"].drop_duplicates()
test_title_2 = test_stage2["title"].drop_duplicates()

In [ ]:
del test_stage2
gc.collect()

In [ ]:
query = pd.concat([train_query,test_query_1,test_query_2],axis=0)
title = pd.concat([train_title,test_title_1,test_title_2],axis=0)
print(query.shape)
print(title.shape)

In [ ]:
query = query.drop_duplicates()
title = title.drop_duplicates()
print(query.shape)
print(title.shape)

In [ ]:
tokenizer = text.Tokenizer(num_words=None)

In [ ]:
tokenizer.fit_on_texts(list(query) + list(title))

In [ ]:
word_index = tokenizer.word_index
print(len(word_index))

In [ ]:
joblib.dump(tokenizer,mypath+"All_Tokenizer.model")

# 三、提取特征

In [ ]:
import pandas as pd
import numpy as np
import os, time
import gc
import lightgbm as lgb
import Levenshtein
import difflib

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import roc_auc_score
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from gensim.models.word2vec import Word2Vec
from nltk.tokenize import word_tokenize
from nltk import ngrams
from joblib import Parallel, delayed
from itertools import chain
from fuzzywuzzy import fuzz

import warnings
warnings.filterwarnings("ignore")

### 训练集和测试集1特征提取

In [ ]:
train = pd.read_csv("/home/kesci/work/train_last_100million.csv")#,nrows=20000000)
print(train.shape)
test = pd.read_csv("/home/kesci/input/bytedance/test_final_part1.csv",names=["query_id","query","query_title_id","title"])
test["title"] = test["title"].strip()
print(test.shape)

In [ ]:
train_feature = pd.DataFrame()
test_feature = pd.DataFrame()

### 1 length、count等统计特征

In [ ]:
# ----- length 特征 -----
for feature in ['query', 'title']:
    print('计算' + feature  + '长度')
    train_feature[feature + "_length"] = train[feature].apply(lambda row : len(row.split()))
train_feature["query_title_abs"] = abs(train_feature["query_length"] - train_feature["title_length"])
train_feature["query_title_ratio"] = train_feature["query_title_abs"]/ (train_feature["query_length"] + train_feature["title_length"])

for feature in ['query', 'title']:
    print('计算' + feature  + '长度')
    test_feature[feature + "_length"] = test[feature].apply(lambda row : len(row.split()))
test_feature["query_title_abs"] = abs(test_feature["query_length"] - test_feature["title_length"])
test_feature["query_title_ratio"] = test_feature["query_title_abs"]/ (test_feature["query_length"] + test_feature["title_length"])

In [ ]:
#---- click 特征 ----
list_click_feature = ['query', 'title']
# 计算某特征单次点击
for feature in list_click_feature:
    print('计算' + feature + '点击次数')
    train_feature[feature + '_click'] = train.groupby(feature)[feature].transform('count')
# 部分二元交叉点击
train_feature['query_title_click'] = train.groupby(['query', 'title']).query.transform('count')
train_feature['title_query_click'] = train.groupby(['query', 'title']).title.transform('count')

##############################################3
for feature in list_click_feature:
    print('计算' + feature + '点击次数')
    test_feature[feature + '_click'] = test.groupby(feature)[feature].transform('count')
# 部分二元交叉点击
test_feature['query_title_click'] = test.groupby(['query', 'title']).query.transform('count')
test_feature['title_query_click'] = test.groupby(['query', 'title']).title.transform('count')

In [ ]:
# ----- nunique 特征 -----
train_feature["query_nunique_title"] = train.groupby("query")["title"].transform("nunique")
train_feature["title_nunique_title"] = train.groupby("title")["query"].transform("nunique")

test_feature["query_nunique_title"] = test.groupby("query")["title"].transform("nunique")
test_feature["title_nunique_title"] = test.groupby("title")["query"].transform("nunique")

In [ ]:
all_feature = pd.concat([train_feature,test_feature],axis=0)

In [ ]:
np.save("/home/kesci/work/drift_data/count_feature_last_100m_split.npy",all_feature)

### 2 编辑距离、fuzzy特征

In [ ]:
test["label"] = -1
all_data = pd.concat([train,test],axis=0)
print(all_data.shape)

In [ ]:
del train,test
gc.collect()

In [ ]:
all_feature = pd.DataFrame()

In [ ]:
def levenshtein_feat(func, query=None, title=None):
    a = query
    b = title
    n = 16
    l = len(a) if isinstance(query, pd.Series) else len(b)
    def tmp(query,title):
        return [func(q, t) for q, t in zip(query, title)]
    res = Parallel(n_jobs=n)(delayed(tmp)(a[int(i*l/n):int((i+1)*l/n)], b[int(i*l/n):int((i+1)*l/n)]) for i in range(n))
    
    return list(chain(*res))

In [ ]:
query = all_data["query"]
title = all_data["title"]

In [ ]:
similarity_func = [Levenshtein.ratio, Levenshtein.distance]

for func in similarity_func:
    print('计算query与title_' + func.__name__  + '相似度')
    all_feature['query_title_levenshtein_' + func.__name__] = levenshtein_feat(func,query,title)

In [ ]:
def fuzz_feat(func, query=None, title=None):
    a = query
    b = title
    n = 16
    l = len(a) if isinstance(query, pd.Series) else len(b)
    def tmp(query,title):
        return [func(q, t) for q, t in zip(query, title)]
    res = Parallel(n_jobs=n)(delayed(tmp)(a[int(i*l/n):int((i+1)*l/n)], b[int(i*l/n):int((i+1)*l/n)]) for i in range(n))
    
    return list(chain(*res))

In [ ]:
fuzz_func = [fuzz.QRatio, fuzz.WRatio, fuzz.partial_ratio, fuzz.partial_token_set_ratio,
                fuzz.partial_token_sort_ratio, fuzz.token_set_ratio, fuzz.token_sort_ratio]
for func in fuzz_func:
    print('计算query与title_' + func.__name__ )
    all_feature['query_title_fuzz_' + func.__name__] = fuzz_feat(func,query,title)

In [ ]:
np.save("/home/kesci/work/drift_data/fuzzy_feature_last_100m.npy",all_feature)

### 3 embedding相似度特征

In [ ]:
all_feature = pd.DataFrame()

In [ ]:
query = train["query"]
title = train["title"]

In [ ]:
def diff_feat(query=None, title=None):
    a = query
    b = title
    n = 14
    l = len(a) if isinstance(query, pd.Series) else len(b)
    def tmp(query,title):
        aa = []
        seq = difflib.SequenceMatcher()
        for q, t in zip(query, title):
            seq.set_seqs(q, t)
            aa.append(seq.ratio())
            
        return aa
    res = Parallel(n_jobs=n)(delayed(tmp)(a[int(i*l/n):int((i+1)*l/n)], b[int(i*l/n):int((i+1)*l/n)]) for i in range(n))
    
    return list(chain(*res))

In [ ]:
all_feature["diff_ratio"] = diff_feat(query, title)
all_feature.shape

In [ ]:
def embedding_feat(func, query=None, title=None,model=None):
    a = query
    b = title
    n = 14
    l = len(a) if isinstance(query, pd.Series) else len(b)
    def tmp(query,title):
        aa = []
        for q, t in zip(query, title):
            row1 = q.split()
            row2 = t.split()
            tokens1 = [token for token in row1 if token in model]
            tokens2 = [token for token in row2 if token in model]
            if len(tokens1) == 0 or len(tokens2) == 0:
                aa.append(0)
                continue
            embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=None).reshape(1, -1)
            embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=None).reshape(1, -1)
            aa.append(func(embedding1,embedding2))
        return aa
    res = Parallel(n_jobs=n)(delayed(tmp)(a[int(i*l/n):int((i+1)*l/n)], b[int(i*l/n):int((i+1)*l/n)]) for i in range(n))
    return list(chain(*res))

In [ ]:
model = Word2Vec.load("/home/kesci/work/drift_data/drift_word2vec_v100.model")

In [ ]:
import time

t0 = time.time()

embedding_func = [braycurtis, cosine, canberra,euclidean, cityblock]
for func in embedding_func:
    print('计算query与title_' + func.__name__ )
    all_feature['query_title_embedding_' + func.__name__] = embedding_feat(func,query,title,model)
    print("Time : ",(time.time() - t0)/60 )
    t0 = time.time()

In [ ]:
np.save("/home/kesci/work/drift_data/stage2_embedding_feature_last_100m.npy",all_feature)

### 测试集1和测试集2提取特征（这里再提取一次测试集1是为了验证和上面顺序保持一致的）

In [ ]:
test_stage1 = pd.read_csv("/home/kesci/input/bytedance/test_final_part1.csv",names=["query_id","query","query_title_id","title"])
test_stage1["title"] = test_stage1["title"].apply(lambda row : row.strip())
print(test_stage1.shape)

In [ ]:
test_stage2 = pd.read_csv("/home/kesci/input/bytedance/bytedance_contest.final_2.csv",names=["query_id","query","query_title_id","title"])
test_stage2["title"] = test_stage2["title"].apply(lambda row : row.strip())
print(test_stage2.shape)

### 1

In [ ]:
test1_feature = pd.DataFrame()
test2_feature = pd.DataFrame()

In [ ]:
# ----- length 特征 -----
for feature in ['query', 'title']:
    print('计算' + feature  + '长度')
    test1_feature[feature + "_length"] = test_stage1[feature].apply(lambda row : len(row.split()))
test1_feature["query_title_abs"] = abs(test1_feature["query_length"] - test1_feature["title_length"])
test1_feature["query_title_ratio"] = test1_feature["query_title_abs"]/ (test1_feature["query_length"] + test1_feature["title_length"])

for feature in ['query', 'title']:
    print('计算' + feature  + '长度')
    test2_feature[feature + "_length"] = test_stage2[feature].apply(lambda row : len(row.split()))
test2_feature["query_title_abs"] = abs(test2_feature["query_length"] - test2_feature["title_length"])
test2_feature["query_title_ratio"] = test2_feature["query_title_abs"]/ (test2_feature["query_length"] + test2_feature["title_length"])

In [ ]:
#---- click 特征 ----
list_click_feature = ['query', 'title']
# 计算某特征单次点击
for feature in list_click_feature:
    print('计算' + feature + '点击次数')
    test1_feature[feature + '_click'] = test_stage1.groupby(feature)[feature].transform('count')
# 部分二元交叉点击
test1_feature['query_title_click'] = test_stage1.groupby(['query', 'title']).query.transform('count')
test1_feature['title_query_click'] = test_stage1.groupby(['query', 'title']).title.transform('count')

##############################################3
for feature in list_click_feature:
    print('计算' + feature + '点击次数')
    test2_feature[feature + '_click'] = test_stage2.groupby(feature)[feature].transform('count')
# 部分二元交叉点击
test2_feature['query_title_click'] = test_stage2.groupby(['query', 'title']).query.transform('count')
test2_feature['title_query_click'] = test_stage2.groupby(['query', 'title']).title.transform('count')

In [ ]:
# ----- nunique 特征 -----
test1_feature["query_nunique_title"] = test_stage1.groupby("query")["title"].transform("nunique")
test1_feature["title_nunique_title"] = test_stage1.groupby("title")["query"].transform("nunique")

test2_feature["query_nunique_title"] = test_stage2.groupby("query")["title"].transform("nunique")
test2_feature["title_nunique_title"] = test_stage2.groupby("title")["query"].transform("nunique")

In [ ]:
all_test_feature = pd.concat([test1_feature,test2_feature],axis=0)

In [ ]:
np.save("/home/kesci/work/drift_data/Test_count_feature.npy",all_test_feature)

### 2

In [ ]:
all_data = pd.concat([test_stage1,test_stage2],axis=0)
print(all_data.shape)

In [ ]:
all_test_feature = pd.DataFrame()

In [ ]:
del test_stage1,test_stage2
gc.collect()

In [ ]:
def levenshtein_feat(func, query=None, title=None):
    a = query
    b = title
    n = 16
    l = len(a) if isinstance(query, pd.Series) else len(b)
    def tmp(query,title):
        return [func(q, t) for q, t in zip(query, title)]
    res = Parallel(n_jobs=n)(delayed(tmp)(a[int(i*l/n):int((i+1)*l/n)], b[int(i*l/n):int((i+1)*l/n)]) for i in range(n))
    
    return list(chain(*res))

In [ ]:
query = all_data["query"]
title = all_data["title"]

In [ ]:
similarity_func = [Levenshtein.ratio, Levenshtein.distance]

for func in similarity_func:
    print('计算query与title_' + func.__name__  + '相似度')
    all_test_feature['query_title_levenshtein_' + func.__name__] = levenshtein_feat(func,query,title)

In [ ]:
def fuzz_feat(func, query=None, title=None):
    a = query
    b = title
    n = 16
    l = len(a) if isinstance(query, pd.Series) else len(b)
    def tmp(query,title):
        return [func(q, t) for q, t in zip(query, title)]
    res = Parallel(n_jobs=n)(delayed(tmp)(a[int(i*l/n):int((i+1)*l/n)], b[int(i*l/n):int((i+1)*l/n)]) for i in range(n))
    
    return list(chain(*res))

In [ ]:
fuzz_func = [fuzz.QRatio, fuzz.WRatio, fuzz.partial_ratio, fuzz.partial_token_set_ratio,
                fuzz.partial_token_sort_ratio, fuzz.token_set_ratio, fuzz.token_sort_ratio]
for func in fuzz_func:
    print('计算query与title_' + func.__name__ )
    all_test_feature['query_title_fuzz_' + func.__name__] = fuzz_feat(func,query,title)

In [ ]:
np.save("/home/kesci/work/drift_data/Test_fuzzy_feature.npy",all_test_feature)

### 3

In [ ]:
all_test_feature = pd.DataFrame()

In [ ]:
def diff_feat(query=None, title=None):
    a = query
    b = title
    n = 14
    l = len(a) if isinstance(query, pd.Series) else len(b)
    def tmp(query,title):
        aa = []
        seq = difflib.SequenceMatcher()
        for q, t in zip(query, title):
            seq.set_seqs(q, t)
            aa.append(seq.ratio())
            
        return aa
    res = Parallel(n_jobs=n)(delayed(tmp)(a[int(i*l/n):int((i+1)*l/n)], b[int(i*l/n):int((i+1)*l/n)]) for i in range(n))
    
    return list(chain(*res))

In [ ]:
all_test_feature["diff_ratio"] = diff_feat(query, title)
all_test_feature.shape

In [ ]:
def embedding_feat(func, query=None, title=None,model=None):
    a = query
    b = title
    n = 14
    l = len(a) if isinstance(query, pd.Series) else len(b)
    def tmp(query,title):
        aa = []
        for q, t in zip(query, title):
            row1 = q.split()
            row2 = t.split()
            tokens1 = [token for token in row1 if token in model]
            tokens2 = [token for token in row2 if token in model]
            if len(tokens1) == 0 or len(tokens2) == 0:
                aa.append(0)
                continue
            embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=None).reshape(1, -1)
            embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=None).reshape(1, -1)
            aa.append(func(embedding1,embedding2))
        return aa
    res = Parallel(n_jobs=n)(delayed(tmp)(a[int(i*l/n):int((i+1)*l/n)], b[int(i*l/n):int((i+1)*l/n)]) for i in range(n))
    return list(chain(*res))

In [ ]:
model = Word2Vec.load("/home/kesci/work/drift_data/drift_word2vec_v100.model")


In [ ]:
# embedding_func = [braycurtis, cosine, canberra,euclidean, cityblock]
# for func in embedding_func:
#     print('计算query与title_' + func.__name__ )
#     all_test_feature['query_title_embedding_' + func.__name__] = embedding_feat(func,query,title,model)
    
import time

t0 = time.time()

embedding_func = [braycurtis, cosine, canberra,euclidean, cityblock]
for func in embedding_func:
    print('计算query与title_' + func.__name__ )
    all_test_feature['query_title_embedding_' + func.__name__] = embedding_feat(func,query,title,model)
    print("Time : ",(time.time() - t0)/60 )
    t0 = time.time()

In [ ]:
np.save("/home/kesci/work/drift_data/Test_embedding_feature.npy",all_test_feature)

# 四、训练lightgbm1，a榜：0.591+

In [ ]:
import pandas as pd
import numpy as np
import os, Levenshtein, time
import gc
import lightgbm as lgb
import difflib
#import swifter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import roc_auc_score
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from gensim.models.word2vec import Word2Vec
from nltk.tokenize import word_tokenize
from nltk import ngrams

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv("/home/kesci/work/train_last_100million.csv")#,nrows=20000000)
print(train.shape)

In [ ]:
labels = train["label"].values

In [ ]:
del train
gc.collect()

In [ ]:
#训练集
count = np.load("/home/kesci/work/drift_data/count_feature_last_100m_split.npy")
print(count.shape)

In [ ]:
#测试集
test_count = np.load("/home/kesci/work/drift_data/Test_count_feature.npy")
print(test_count.shape)

In [ ]:
#测试集
test_count = np.load("/home/kesci/work/drift_data/Test_count_feature.npy")
print(test_count.shape)

In [ ]:
#测试集
test_fuzzy = np.load("/home/kesci/work/drift_data/Test_fuzzy_feature.npy")
print(test_fuzzy.shape)

In [ ]:
#新训练集 (没有diff_ratio)
embed = np.load("/home/kesci/work/drift_data/stage2_embedding_feature_last_100m.npy")
print(embed.shape)

In [ ]:
#测试集
test_embed = np.load("/home/kesci/work/drift_data/Test_embedding_feature.npy")
print(test_embed.shape)

In [ ]:
#合并特征

# train
train = np.hstack((count[:100000000],fuzzy[:100000000]))
print(train.shape)
train = np.hstack((train,embed))
print(train.shape)

In [ ]:
# valid
test1 = np.hstack((test_count[:20000000],test_fuzzy[:20000000]))
test1 = np.hstack((test1,test_embed[:20000000]))
print(test1.shape)

In [ ]:
test2 = np.hstack((test_count[20000000:],test_fuzzy[20000000:]))
test2 = np.hstack((test2,test_embed[20000000:]))
print(test2.shape)

In [ ]:
# 划分训练集与验证集
valid_feat = train[90000000:]
train_feat = train[:90000000]
valid_label = labels[90000000:]
train_label = labels[:90000000]

In [ ]:
clf = lgb.LGBMClassifier(
        boosting_type = 'gbdt', num_leaves = 64, reg_alpha = 5, reg_lambda = 5,
        n_estimators = 1500, objective = 'binary',subsample = 0.7, colsample_bytree = 0.7, 
        subsample_freq = 1,learning_rate = 0.05, random_state = 2019,
        n_jobs=14)

clf.fit(train_feat, train_label, eval_set = [(train_feat, train_label),(valid_feat,valid_label)],
        early_stopping_rounds = 100,verbose = 100)

In [ ]:
cols=[
        "query_length","title_length","query_title_abs","query_title_ratio","query_click","title_click",
        "query_title_click","title_query_click","query_nunique_title","title_nunique_title",
        
        "levenshtein_ratio","levenshtein_distence","fuzz_QRatio","fuzz_WRatio",
        "fuzz_partial_ratio","fuzz_partial_token_set_ratio","fuzz_partial_token_sort_ratio",
        "fuzz_token_set_ratio","fuzz_token_sort_ratio",
        
        "diff_ratio","embedding_braycurtis","embedding_cosine","embedding_canberra","embedding_euclidean",
        "embedding_cityblock"]

In [ ]:
feature_importance = pd.DataFrame()
feature_importance["feature"] = cols
feature_importance["importances"] = clf.feature_importances_
feature_importance.sort_values("importances",ascending=False)

In [ ]:
# 保存模型
from sklearn.externals import joblib

joblib.dump(clf,'/home/kesci/work/drift_data/dump_lgb_model.pkl')
clf = joblib.load('/home/kesci/work/drift_data/dump_lgb_model.pkl')

In [ ]:
#valid 0.691
preds = clf.predict_proba(valid)[:,1]
roc_auc_score(valid_label,preds)

### 测试集1

In [ ]:
probs = clf.predict_proba(test1)[:,1]

In [ ]:
path = "/home/kesci/input/bytedance/"
test_path = path+"test_final_part1.csv"
test = pd.read_csv(test_path,header=None,names=["query_id","query","query_title_id","title"])
print("ok")

In [ ]:
submit = test[["query_id","query_title_id"]]
submit["prediction"] = probs
submit.to_csv("stage1_lgb_submit_v0.csv",index=False,header=None)

### 测试集2

In [ ]:
probs = clf.predict_proba(test2)[:,1]

In [ ]:
test = pd.read_csv("/home/kesci/input/bytedance/bytedance_contest.final_2.csv",header=None,names=["query_id","query","query_title_id","title"])
print("ok")

In [ ]:
submit = test[["query_id","query_title_id"]]
submit["prediction"] = probs
submit.to_csv("stage2_lgb_submit_v0.csv",index=False,header=None)

# 五、nn1 Textcnn模型，a榜：0.603+

In [ ]:
import pandas as pd
import numpy as np
import keras 
import gc

from tqdm import tqdm
from gensim.models.word2vec import Word2Vec
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.callbacks import *
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.externals import joblib

import warnings
warnings.filterwarnings("ignore")

In [ ]:
path = "/home/kesci/input/bytedance/"
mypath = "/home/kesci/work/drift_data/"

batch_size = 256
num_words = 700000
embed_size = 300
max_sequence_length = 20

In [ ]:
train = pd.read_csv("/home/kesci/work/train_last_100million.csv")#,nrows=20000000)
print(train.shape)
test1 = pd.read_csv("/home/kesci/input/bytedance/test_final_part1.csv",header=None,names=["query_id","query","query_title_id","title"])
test1["title"] = test1["title"].apply(lambda row : row.strip())
print(test1.shape)

In [ ]:
tokenizer = joblib.load("/home/kesci/work/drift_data/All_Tokenizer.model")


In [ ]:
import time
t0 = time.time()

'''
Time :  27.89003413915634
Time :  38.88456140359243
'''

train_query = sequence.pad_sequences(tokenizer.texts_to_sequences(list(train["query"])), maxlen=max_sequence_length)
print("Time : ",(time.time() -t0)/60)
t0 = time.time()
train_title = sequence.pad_sequences(tokenizer.texts_to_sequences(list(train["title"])), maxlen=max_sequence_length)
print("Time : ",(time.time() -t0)/60)
t0 = time.time()
test_query = sequence.pad_sequences(tokenizer.texts_to_sequences(list(test1["query"])), maxlen=max_sequence_length)
print("Time : ",(time.time() -t0)/60)
t0 = time.time()
test_title = sequence.pad_sequences(tokenizer.texts_to_sequences(list(test1["title"])), maxlen=max_sequence_length)
print("Time : ",(time.time() -t0)/60)
t0 = time.time()

In [ ]:
labels = train["label"].values

In [ ]:
valid_query = train_query[90000000:]
valid_title = train_title[90000000:]

train_query = train_query[:90000000]
train_title = train_title[:90000000]

train_labels = labels[:90000000]
valid_labels = labels[90000000:]

In [ ]:
train_y = to_categorical(train_labels)
valid_y = to_categorical(valid_labels)

In [ ]:
word_index = tokenizer.word_index
print(len(word_index))
num_words = len(word_index)#min(num_words, len(word_index))
print(num_words)

In [ ]:
model = Word2Vec.load("/home/kesci/work/drift_data/drift_word2vec_v100.model")

In [ ]:
ss = 0
embedding_matrix = np.zeros((num_words+1, embed_size))
print(len(word_index.items()))
for word, i in word_index.items():
    if word in model.wv.vocab:
        ss += 1
        if i >= num_words:
            break
        embedding_matrix[i] = model.wv[word]
    else:
        unk_vec = np.random.random(embed_size) * 0.5
        unk_vec = unk_vec - unk_vec.mean()
        embedding_matrix[i] = unk_vec
print(ss)
print(ss/num_words)
print(embedding_matrix.shape)

In [ ]:
#训练集特征读取
#训练集
count = np.load("/home/kesci/work/drift_data/count_feature_last_100m_split.npy")
print(count.shape)
train_count = count[:100000000]
del count
gc.collect()

#训练集
fuzzy = np.load("/home/kesci/work/drift_data/fuzzy_feature_last_100m.npy")
fuzzy.shape
train_fuzzy = fuzzy[:100000000]
print(train_fuzzy.shape)
del fuzzy
gc.collect()

embed = np.load("/home/kesci/work/drift_data/stage2_embedding_feature_last_100m.npy")
print(embed.shape)
train_embed = embed[:100000000]
print(train_embed.shape)
del embed
gc.collect()

diff_ratio = np.load("/home/kesci/work/drift_data/diff_ratio_feature_100m.npy")
diff_ratio.shape
train_diff = diff_ratio[:100000000]
del diff_ratio
gc.collect()

train_embed = np.hstack((train_diff,train_embed))
print(train_embed.shape)


train = np.hstack((train_count,train_fuzzy))
print(train.shape)
train = np.hstack((train,train_embed))
print(train.shape)

del train_count,train_fuzzy,train_embed
gc.collect()
#np.save("/home/kesci/work/drift_data/demo_train_feat.npy",train)

In [ ]:
# 测试集test1
test_count = np.load("/home/kesci/work/drift_data/Test_count_feature.npy")
print(test_count.shape)
test_fuzzy = np.load("/home/kesci/work/drift_data/Test_fuzzy_feature.npy")
print(test_fuzzy.shape)
valid = np.hstack((test_count[:20000000],test_fuzzy[:20000000]))
del test_count,test_fuzzy
gc.collect()

test_embed = np.load("/home/kesci/work/drift_data/Test_embedding_feature.npy")
print(test_embed.shape)

# valid
#valid = np.hstack((test_count[:20000000],test_fuzzy[:20000000]))
valid = np.hstack((valid,test_embed[:20000000]))
print(valid.shape)
del test_embed
gc.collect()
#np.save("/home/kesci/work/drift_data/demo_valid_feat.npy",valid)

In [ ]:
train_feat = train[:90000000]
valid_feat = train[90000000:]
print(train_feat.shape)
print(valid_feat.shape)

In [ ]:
test_feat = valid
print(test_feat.shape)

In [ ]:
feat_shape = train_feat.shape[1]
feat_shape

In [ ]:
def build_model(emb_matrix):
    
    embed = Embedding(
        input_dim=emb_matrix.shape[0],
        output_dim=emb_matrix.shape[1],
        weights=[emb_matrix],
        input_length=max_sequence_length,
        trainable=False)

    conv1 = Conv1D(filters=128, kernel_size=1, padding='same', activation='relu')
    conv2 = Conv1D(filters=128, kernel_size=2, padding='same', activation='relu')
    conv3 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')
    conv4 = Conv1D(filters=128, kernel_size=4, padding='same', activation='relu')
    conv5 = Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')
    conv6 = Conv1D(filters=32, kernel_size=6, padding='same', activation='relu')

    seq1 = Input(shape=(max_sequence_length,))
    seq2 = Input(shape=(max_sequence_length,))
    
    emb1 = embed(seq1)
    emb2 = embed(seq2)

    conv1a = conv1(emb1)
    glob1a = GlobalAveragePooling1D()(conv1a)
    max1a = GlobalMaxPooling1D()(conv1a)

    conv1b = conv1(emb2)
    glob1b = GlobalAveragePooling1D()(conv1b)
    max1b = GlobalMaxPooling1D()(conv1b)

    conv2a = conv2(emb1)
    glob2a = GlobalAveragePooling1D()(conv2a)
    max2a = GlobalMaxPooling1D()(conv2a)

    conv2b = conv2(emb2)
    glob2b = GlobalAveragePooling1D()(conv2b)
    max2b = GlobalMaxPooling1D()(conv2b)

    conv3a = conv3(emb1)
    glob3a = GlobalAveragePooling1D()(conv3a)
    max3a = GlobalMaxPooling1D()(conv3a)

    conv3b = conv3(emb2)
    glob3b = GlobalAveragePooling1D()(conv3b)
    max3b = GlobalMaxPooling1D()(conv3b)

    conv4a = conv4(emb1)
    glob4a = GlobalAveragePooling1D()(conv4a)
    max4a = GlobalMaxPooling1D()(conv4a)

    conv4b = conv4(emb2)
    glob4b = GlobalAveragePooling1D()(conv4b)
    max4b = GlobalMaxPooling1D()(conv4b)

    conv5a = conv5(emb1)
    glob5a = GlobalAveragePooling1D()(conv5a)
    max5a = GlobalMaxPooling1D()(conv5a)
  
    conv5b = conv5(emb2)
    glob5b = GlobalAveragePooling1D()(conv5b)
    max5b = GlobalMaxPooling1D()(conv5b)

    conv6a = conv6(emb1)
    glob6a = GlobalAveragePooling1D()(conv6a)
    max6a = GlobalMaxPooling1D()(conv6a)

    conv6b = conv6(emb2)
    glob6b = GlobalAveragePooling1D()(conv6b)
    max6b = GlobalMaxPooling1D()(conv6b)

    mergea = concatenate([glob1a, glob2a, glob3a, glob4a, glob5a, glob6a])
    mergeb = concatenate([glob1b, glob2b, glob3b, glob4b, glob5b, glob6b])
    mergec = concatenate([max1a, max2a, max3a, max4a, max5a, max6a])
    merged = concatenate([max1b, max2b, max3b, max4b, max5b, max6b])

    diff_ave = Lambda(lambda x: K.abs(x[0] - x[1]), output_shape=(4 * 128 + 2*32,))([mergea, mergeb])
    mul_ave = Lambda(lambda x: x[0] * x[1], output_shape=(4 * 128 + 2*32,))([mergea, mergeb])
    diff_max = Lambda(lambda x: K.abs(x[0] - x[1]), output_shape=(4 * 128 + 2*32,))([mergec, merged])
    mul_max = Lambda(lambda x: x[0] * x[1], output_shape=(4 * 128 + 2*32,))([mergec, merged])
    
    #feature 输入
    feature_input = Input(shape=(feat_shape,))
    feature_dense = Dense(128, activation='relu')(feature_input)
    feature_dense = Dropout(0.2)(feature_dense)
    feature_dense = Dense(128, activation='relu')(feature_dense)
    feature_dense = BatchNormalization()(feature_dense)

    merge = concatenate([diff_ave, mul_ave,diff_max,mul_max,feature_dense])
    
    print(merge.shape)
    x = BatchNormalization()(merge)
    x = Dense(300, activation='relu')(x)
    x = Dropout(0.2)(x)
    pred = Dense(2, activation='softmax')(x)

    model = Model(inputs=[seq1, seq2,feature_input], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
 
    return model

In [ ]:
ft = build_model(embedding_matrix)

early_stop = EarlyStopping(patience=3)

check_point = ModelCheckpoint('/home/kesci/work/drift_data/models/Stage2_Improved_TextCNN_Epoch_{epoch:02d}.hdf5', monitor="val_loss", 
                    mode="min", save_best_only=False, verbose=1)

In [ ]:
 hist = ft.fit([train_query, train_title,train_feat],
                     train_y, 
                     validation_data=([valid_query, valid_title,valid_feat], valid_y),
                     epochs=1,
                     batch_size=512,
                     shuffle=True,
                     callbacks=[early_stop, check_point])

In [ ]:
ft.load_weights("/home/kesci/work/drift_data/models/Stage2_Improved_TextCNN_Epoch_01.hdf5")

In [ ]:
preds_1 = ft.predict([valid_query,valid_title,valid_feat], batch_size=512, verbose=1)[:,1]
roc_auc_score(valid_labels,preds_1)

In [ ]:
test_probs = ft.predict([test_query,test_title,test_feat],batch_size=512*4,verbose=1)

### 测试集2预测

In [ ]:
# 测试集test2
test_count = np.load("/home/kesci/work/drift_data/Test_count_feature.npy")
print(test_count.shape)
test_fuzzy = np.load("/home/kesci/work/drift_data/Test_fuzzy_feature.npy")
print(test_fuzzy.shape)
test2_feat = np.hstack((test_count[20000000:],test_fuzzy[20000000:]))
del test_count,test_fuzzy
gc.collect()

test_embed = np.load("/home/kesci/work/drift_data/Test_embedding_feature.npy")
print(test_embed.shape)

# valid
#valid = np.hstack((test_count[:20000000],test_fuzzy[:20000000]))
test2_feat = np.hstack((test2_feat,test_embed[20000000:]))
print(test2_feat.shape)
del test_embed
gc.collect()

In [ ]:
t0 = time.time()
test2_query = sequence.pad_sequences(tokenizer.texts_to_sequences(list(test2["query"])), maxlen=max_sequence_length)
print("Time : ",(time.time() -t0)/60)
t0 = time.time()
test2_title = sequence.pad_sequences(tokenizer.texts_to_sequences(list(test2["title"])), maxlen=max_sequence_length)
print("Time : ",(time.time() -t0)/60)

np.save(mypath+"test2_query.npy",test2_query)
np.save(mypath+"test2_title.npy",test2_title)

test2_query = np.load(mypath+"test2_query.npy")
test2_title = np.load(mypath+"test2_title.npy")

In [ ]:
test2_probs = ft.predict([test2_query,test2_title,test2_feat],batch_size=512*4,verbose=1)

In [ ]:
test2 = pd.read_csv("/home/kesci/input/bytedance/bytedance_contest.final_2.csv",header=None,names=["query_id","query","query_title_id","title"])

In [ ]:
submit = test2[["query_id","query_title_id"]]
submit["probs"] = test2_probs[:,1]
submit.to_csv("stage2_improved_textcnn_feat_submit_v0.csv",index=False,header=None)

#  六、nn2 BiLstm_Fasttext 模型，a榜：0.603-0.605（没有线上提交）

In [ ]:
#bilstm_fasttext
def model3(embedding_matrix):
     
    query = Input(shape=(max_sequence_length,),name="query")
    title = Input(shape=(max_sequence_length,),name="title")
    
    embedding = Embedding(input_dim=embedding_matrix.shape[0],
                            output_dim=embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=False,name="embedding")
    
    q_embed = embedding(query)
    q_embed = SpatialDropout1D(0.2)(q_embed)
    
    print(q_embed.shape)
    
    t_embed = embedding(title)
    t_embed = SpatialDropout1D(0.2)(t_embed)
    
    print(t_embed.shape)
    
    share_BiLSTM1 = Bidirectional(CuDNNLSTM(100, return_sequences=True))
    share_BiLSTM2= Bidirectional(CuDNNLSTM(100, return_sequences=True))
    
    # 两层 Bilstm
    q_lstm1 = share_BiLSTM1(q_embed)
    q_lstm = share_BiLSTM2(q_lstm1)
    
    q_lstm = concatenate([q_embed, q_lstm1, q_lstm])
    print("q_lstm : ",q_lstm.shape)
    
    t_lstm1 = share_BiLSTM1(t_embed)
    t_lstm = share_BiLSTM2(t_lstm1)
    
    t_lstm =  concatenate([t_embed, t_lstm1, t_lstm])
    
    q_max = GlobalMaxPool1D()(q_lstm)
    q_ave = GlobalAveragePooling1D()(q_lstm)
    t_max = GlobalMaxPool1D()(t_lstm)
    t_ave = GlobalAveragePooling1D()(t_lstm)
        
    print(q_max.shape)
    print(q_ave.shape)
    print(t_max.shape)
    print(t_ave.shape)
    
    # 取差和积
    maxpool_diff = Lambda(lambda x: K.abs(x[0] - x[1]), output_shape=(700,))([q_max, t_max])
    maxpool_mul = Lambda(lambda x: x[0] * x[1], output_shape=(700 ,))([q_max, t_max])
    
    avepool_diff = Lambda(lambda x: K.abs(x[0] - x[1]), output_shape=(700,))([q_ave, t_ave])
    avepool_mul = Lambda(lambda x: x[0] * x[1], output_shape=(700 ,))([q_ave, t_ave])
    
    #feature 输入
    feature_input = Input(shape=(feat_shape,))
    feature_dense = Dense(128, activation='relu')(feature_input)
    feature_dense = Dropout(0.2)(feature_dense)
    feature_dense = Dense(128, activation='relu')(feature_dense)
    feature_dense = BatchNormalization()(feature_dense)
    
    merged = concatenate([ maxpool_diff,maxpool_mul ,avepool_diff , avepool_mul,feature_dense])
    print(merged.shape)
    
    x = BatchNormalization()(merged)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    pred = Dense(2, activation='softmax')(x)
    
    model = Model(inputs=[query, title,feature_input], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    
    return model

In [ ]:
model = model3(embedding_matrix)

In [ ]:
early_stop = EarlyStopping(patience=3)
check_point = ModelCheckpoint('/home/kesci/work/drift_data/models/Stage2_BiLSTM_FastText_Epoch1.hdf5', monitor="val_loss", 
                    mode="min", save_best_only=True, verbose=1)

In [ ]:
 history = model.fit([train_query, train_title,train_feat],train_y, 
                      validation_data=([valid_query, valid_title,valid_feat], valid_y),
                      epochs=1,batch_size=512,shuffle=True,
                      callbacks=[early_stop, check_point])

### 测试集2

In [ ]:
test2_probs = model.predict([test2_query,test2_title,test2_feat],batch_size=512*4,verbose=1)

In [ ]:
submit = test2[["query_id","query_title_id"]]
submit["probs"] = test2_probs[:,1]
submit.to_csv("stage2_LSTM_FastText_feat_submit_Epoch1.csv",index=False,header=None)

# 七、nn3 BiLstm_TextCNN 模型，a榜：0.610+

In [ ]:
# 模型2 BiLSTM—CNN 
def model2(embedding_matrix):
     
    query = Input(shape=(max_sequence_length,),name="query")
    title = Input(shape=(max_sequence_length,),name="title")
    
    embedding = Embedding(input_dim=embedding_matrix.shape[0],
                            output_dim=embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=False,name="embedding")
    
    q_embed = embedding(query)
    q_embed = SpatialDropout1D(0.2)(q_embed)
    
    print(q_embed.shape)
    
    t_embed = embedding(title)
    t_embed = SpatialDropout1D(0.2)(t_embed)
    
    print(t_embed.shape)
    
    share_BiLSTM1 = Bidirectional(CuDNNLSTM(100, return_sequences=True))
    share_BiLSTM2= Bidirectional(CuDNNLSTM(100, return_sequences=True))
    
    # 两层 Bilstm
    q_lstm1 = share_BiLSTM1(q_embed)
    q_lstm = share_BiLSTM2(q_lstm1)
    
    q_lstm = concatenate([q_embed, q_lstm1, q_lstm])
    print("q_lstm : ",q_lstm.shape)
    
    t_lstm1 = share_BiLSTM1(t_embed)
    t_lstm = share_BiLSTM2(t_lstm1)
    
    t_lstm =  concatenate([t_embed, t_lstm1, t_lstm])
    
    # 卷积层
    conv1 = Conv1D(filters=128, kernel_size=1, padding='same', activation='relu')
    conv2 = Conv1D(filters=128, kernel_size=2, padding='same', activation='relu')
    conv3 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')
    conv4 = Conv1D(filters=128, kernel_size=4, padding='same', activation='relu')
    conv5 = Conv1D(filters=128, kernel_size=5, padding='same', activation='relu')

    # 1
    q_conv1 = conv1(q_lstm)
    q_maxpool1 = GlobalMaxPool1D()(q_conv1)
    q_avepool1 = GlobalAveragePooling1D()(q_conv1)
    
    q_conv2 = conv2(q_lstm)
    q_maxpool2 = GlobalMaxPool1D()(q_conv2)
    q_avepool2 = GlobalAveragePooling1D()(q_conv2)
    
    q_conv3 = conv3(q_lstm)
    q_maxpool3 = GlobalMaxPool1D()(q_conv3)
    q_avepool3 = GlobalAveragePooling1D()(q_conv3)
    
    q_conv4 = conv4(q_lstm)
    q_maxpool4 = GlobalMaxPool1D()(q_conv4)
    q_avepool4 = GlobalAveragePooling1D()(q_conv4)
    
    q_conv5 = conv5(q_lstm)
    q_maxpool5 = GlobalMaxPool1D()(q_conv5)
    q_avepool5 = GlobalAveragePooling1D()(q_conv5)
    
    print("conv1 : ", q_conv1.shape)
    print("maxpool : ",q_maxpool1.shape)
    print("avepool : ",q_avepool1.shape)
    
    q_max = concatenate([q_maxpool1,q_maxpool2,q_maxpool3,q_maxpool4,q_maxpool5])
    q_ave = concatenate([q_avepool1,q_avepool2,q_avepool3,q_avepool4,q_avepool5])
    
    print("query maxpooling : ",q_max.shape)
    print("query averagepooling : ",q_ave.shape)
    
    t_conv1 = conv1(t_lstm)
    t_maxpool1 = GlobalMaxPool1D()(t_conv1)
    t_avepool1 = GlobalAveragePooling1D()(t_conv1)
        
    t_conv2 = conv2(t_lstm)
    t_maxpool2 = GlobalMaxPool1D()(t_conv2)
    t_avepool2 = GlobalAveragePooling1D()(t_conv2)
        
    t_conv3 = conv3(t_lstm)
    t_maxpool3 = GlobalMaxPool1D()(t_conv3)
    t_avepool3 = GlobalAveragePooling1D()(t_conv3)
        
    t_conv4 = conv4(t_lstm)
    t_maxpool4 = GlobalMaxPool1D()(t_conv4)
    t_avepool4 = GlobalAveragePooling1D()(t_conv4)
        
    t_conv5 = conv5(t_lstm)
    t_maxpool5 = GlobalMaxPool1D()(t_conv5)
    t_avepool5 = GlobalAveragePooling1D()(t_conv5)
        
    #maxpool 和 average 合并
    q_max = concatenate([q_maxpool1,q_maxpool2,q_maxpool3,q_maxpool4,q_maxpool5])
    q_ave = concatenate([q_avepool1,q_avepool2,q_avepool3,q_avepool4,q_avepool5])
    
    t_max = concatenate([t_maxpool1,t_maxpool2,t_maxpool3,t_maxpool4,t_maxpool5])
    t_ave = concatenate([t_avepool1,t_avepool2,t_avepool3,t_avepool4,t_avepool5])
        
    # 取差和积
    maxpool_diff = Lambda(lambda x: K.abs(x[0] - x[1]), output_shape=(5 * 128,))([q_max, t_max])
    maxpool_mul = Lambda(lambda x: x[0] * x[1], output_shape=(5 * 128 ,))([q_max, t_max])
    
    avepool_diff = Lambda(lambda x: K.abs(x[0] - x[1]), output_shape=(5 * 128,))([q_ave, t_ave])
    avepool_mul = Lambda(lambda x: x[0] * x[1], output_shape=(5 * 128 ,))([q_ave, t_ave])
    
    #feature 输入
    feature_input = Input(shape=(feat_shape,))
    feature_dense = Dense(128, activation='relu')(feature_input)
    feature_dense = Dropout(0.2)(feature_dense)
    feature_dense = Dense(128, activation='relu')(feature_dense)
    feature_dense = BatchNormalization()(feature_dense)
    
    merged = concatenate([ maxpool_diff,maxpool_mul ,avepool_diff , avepool_mul,feature_dense])
    print(merged.shape)
    
    
    x = BatchNormalization()(merged)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    pred = Dense(2, activation='softmax')(x)
    
    model = Model(inputs=[query, title,feature_input], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    
    return model

In [ ]:
model = model2(embedding_matrix)

In [ ]:
early_stop = EarlyStopping(patience=3)
check_point = ModelCheckpoint('/home/kesci/work/drift_data/models/Stage2_LSTM_CNN_Epoch_{epoch:02d}.hdf5', monitor="val_loss", 
                    mode="min", save_best_only=True, verbose=1)

In [ ]:
 history = model.fit([train_query, train_title,train_feat],train_y, 
                      validation_data=([valid_query, valid_title,valid_feat], valid_y),
                      epochs=3,batch_size=512,shuffle=True,
                      callbacks=[early_stop, check_point])

In [ ]:
preds_1 = model.predict([valid_query,valid_title,valid_feat], batch_size=512, verbose=1)[:,1]
roc_auc_score(valid_labels,preds_1)

In [ ]:
test2_probs = model.predict([test2_query,test2_title,test2_feat],batch_size=512*4,verbose=1)

In [ ]:
submit = test2[["query_id","query_title_id"]]
submit["probs"] = test2_probs[:,1]
submit.to_csv("stage2_LSTM_CNN_feat_submit_Epoch3.csv",index=False,header=None)

# 八、lightgbm2 a榜：0.590+

In [2]:
# 显示cell运行时长
%load_ext klab-autotime

In [3]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple fuzzywuzzy
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple pyemd
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple pandas --upgrade

Requirement already up-to-date: pandas in /opt/conda/lib/python3.6/site-packages
Requirement already up-to-date: python-dateutil>=2.6.1 in /opt/conda/lib/python3.6/site-packages (from pandas)
Requirement already up-to-date: pytz>=2017.2 in /opt/conda/lib/python3.6/site-packages (from pandas)
Requirement already up-to-date: numpy>=1.13.3 in /opt/conda/lib/python3.6/site-packages (from pandas)
Requirement already up-to-date: six>=1.5 in /opt/conda/lib/python3.6/site-packages (from python-dateutil>=2.6.1->pandas)
time: 20.4 s


In [4]:
import os
import csv
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import gensim
from gensim import corpora, models
from collections import Counter
from itertools import chain
from array import array
from fuzzywuzzy import fuzz
from tqdm import tqdm
from scipy.linalg import norm
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra
from nltk import word_tokenize
from joblib import Parallel, delayed
from gensim.models import Word2Vec
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import lightgbm as lgb

time: 3.06 s


In [5]:
# tf相似度
def my_tf_similarity(s1, s2):
    c1 = Counter(s1)
    c2 = Counter(s2)
    union = set(s1) | set(s2)
    vectors = np.zeros((2, len(union)))
    for i, w in enumerate(union):
        vectors[0, i] = c1[w]
        vectors[1, i] = c2[w]
    return np.dot(vectors[0], vectors[1]) / (norm(vectors[0]) * norm(vectors[1]))

time: 1.65 ms


In [6]:
# 杰卡德系数1
def my_jaccard1(s1, s2):
    """
    考虑出现次数
    """
    a = set(s1)
    b = set(s2)
    numerator = denominator = 0
    for i in a & b:
        c1, c2 = s1.count(i), s2.count(i)
        numerator += c1 if c1 < c2 else c2 
    for i in a | b:
        c1, c2 = s1.count(i), s2.count(i)
        denominator += c1 if c1 > c2 else c2 
    return numerator / denominator

time: 3.52 ms


In [7]:
# 杰卡德系数2
def my_jaccard2(s1, s2):
    """
    不考虑出现次数
    """    
    a = set(s1)
    b = set(s2)
    return len(a & b) / len(a | b)

time: 1.03 ms


In [8]:
# 编辑距离
def levenshtein(seq1, seq2, max_dist=-1):
    """
    seq1, seq2:空格分隔的句子
    """
    if seq1 == seq2:
        return 0
    len1, len2 = len(seq1), len(seq2)
    # if max_dist >= 0 and abs(len1 - len2) > max_dist:
    #     return -1
    # if len1 == 0:
    #     return len2
    # if len2 == 0:
    #     return len1
    if len1 < len2:
        len1, len2 = len2, len1
        seq1, seq2 = seq2, seq1
    
    column = array('L', range(len2 + 1))
    
    for x in range(1, len1 + 1):
        column[0] = x
        last = x - 1
        for y in range(1, len2 + 1):
            old = column[y]
            cost = int(seq1[x - 1] != seq2[y - 1])
            column[y] = min(column[y] + 1, column[y - 1] + 1, last + cost)
            last = old
        if max_dist >= 0 and min(column) > max_dist:
            return -1
    
    if max_dist >= 0 and column[len2] > max_dist:
        # stay consistent, even if we have the exact distance
        return -1
    return column[len2]

time: 2.99 ms


In [9]:
#sorensen 相似性系数
def sorensen(seq1, seq2):
	set1, set2 = set(seq1), set(seq2)
	return 1 - (2 * len(set1 & set2) / float(len(set1) + len(set2)))

time: 1.2 ms


In [10]:
# 生成句向量
def sent2vec(words):
    M = []
    for w in words:
        try:
            M.append(model.wv[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / len(M)

time: 1.09 ms


In [11]:
def qauc(x):
    try:
        score = roc_auc_score(x.label, x.pred)
    except:
        score = -1
    return score

time: 1.01 ms


In [12]:
# title中包含的query中的词数
def n_include_word(s1, s2):
    n = 0
    for w in s1:
        if w in s2:
            n += 1
    return n

time: 1.02 ms


In [13]:
# 最长子串长度
def find_lcsubstr(s1, s2): 
	m=[[0 for i in range(len(s2)+1)]  for j in range(len(s1)+1)]  #生成0矩阵，为方便后续计算，比字符串长度多了一列
	mmax=0   #最长匹配的长度
	p=0  #最长匹配对应在s1中的最后一位
	for i in range(len(s1)):
		for j in range(len(s2)):
			if s1[i]==s2[j]:
				m[i+1][j+1]=m[i][j]+1
				if m[i+1][j+1]>mmax:
					mmax=m[i+1][j+1]
					p=i+1
	return mmax   #返回最长子串及其长度 

time: 2.15 ms


In [14]:
# 最长子序列长度
def find_lcseque(s1, s2): 
	# 生成字符串长度加1的0矩阵，m用来保存对应位置匹配的结果
	m = [ [ 0 for x in range(len(s2)+1) ] for y in range(len(s1)+1) ] 
	# d用来记录转移方向
	d = [ [ None for x in range(len(s2)+1) ] for y in range(len(s1)+1) ] 
    
	for p1 in range(len(s1)): 
		for p2 in range(len(s2)): 
			if s1[p1] == s2[p2]:            #字符匹配成功，则该位置的值为左上方的值加1
				m[p1+1][p2+1] = m[p1][p2]+1
				d[p1+1][p2+1] = 'ok'          
			elif m[p1+1][p2] > m[p1][p2+1]:  #左值大于上值，则该位置的值为左值，并标记回溯时的方向
				m[p1+1][p2+1] = m[p1+1][p2] 
				d[p1+1][p2+1] = 'left'          
			else:                           #上值大于左值，则该位置的值为上值，并标记方向up
				m[p1+1][p2+1] = m[p1][p2+1]   
				d[p1+1][p2+1] = 'up'         
	(p1, p2) = (len(s1), len(s2)) 
	s = [] 
	while m[p1][p2]:    #不为None时
		c = d[p1][p2]
		if c == 'ok':   #匹配成功，插入该字符，并向左上角找下一个
			s.append(s1[p1-1])
			p1-=1
			p2-=1 
		if c =='left':  #根据标记，向左找下一个
			p2 -= 1
		if c == 'up':   #根据标记，向上找下一个
			p1 -= 1
	s.reverse() 
	return len(s) 

time: 3.9 ms


In [16]:
# 多进程并行提取特征
def extract_feat(func, query=None, title=None, sent2vec=None):
    a = query
    b = title
    n = 16
    l = len(a) if isinstance(query, pd.Series) else len(b)
    def tmp1(query, title):
        return [func(q.split(), t.split()) for q, t in zip(query, title)]
    def tmp2(query):
        return [func(q.split()) for q in query]
    def tmp3(title):
        return [func(t.split()) for t in title]
    def tmp4(query, title):
        return [func(sent2vec(q.split()), sent2vec(t.split())) for q, t in zip(query, title)]
    def tmp5(query):
        return [func(sent2vec(q.split())) for q in query]
    def tmp6(title):
        return [func(sent2vec(t.split())) for t in title]
    if isinstance(query, pd.Series) and isinstance(title, pd.Series):
        tmp = tmp4 if sent2vec else tmp1
        res = Parallel(n_jobs=n)(delayed(tmp)(a[int(i*l/n):int((i+1)*l/n)], b[int(i*l/n):int((i+1)*l/n)]) for i in range(n))
    elif isinstance(query, pd.Series):
        tmp = tmp5 if sent2vec else tmp2
        res = Parallel(n_jobs=n)(delayed(tmp)(query[int(i*l/n):int((i+1)*l/n)]) for i in range(n))
    elif isinstance(title, pd.Series):
        tmp = tmp6 if sent2vec else tmp3
        res = Parallel(n_jobs=n)(delayed(tmp)(title[int(i*l/n):int((i+1)*l/n)]) for i in range(n))
    return list(chain(*res))

time: 4.69 ms


In [23]:
# 最终测试集
test = pd.read_csv("/home/kesci/input/bytedance/bytedance_contest.final_2.csv", 
                    names=['query_id','query','query_title_id','title'])

time: 2min 56s


In [19]:
# 前5kw训练集
train = pd.read_csv("/home/kesci/input/bytedance/train_final.csv", nrows=50000000,
                    names=['query_id','query','query_title_id','title', 'label'])

time: 917 µs


In [27]:
query = pd.concat([train['query'], test['query']], ignore_index=True)
title = pd.concat([train.title, test.title], ignore_index=True)

time: 2.09 s


In [28]:
feat = pd.DataFrame()

time: 1.1 ms


In [29]:
# query长度
feat['len_query'] = extract_feat(len, query=query)
# title长度
feat['len_title'] = extract_feat(len, title=title)
feat['diff_len'] = feat['len_title'] - feat['len_query']
# 杰卡德系数
feat['jaccard1'] = extract_feat(my_jaccard1, query, title)
feat['jaccard2'] = extract_feat(my_jaccard2, query, title)
# tf相似度
feat['tf_similarity'] = extract_feat(my_tf_similarity, query, title)
# 编辑距离
feat['levenshtein'] = extract_feat(levenshtein, query, title)

time: 21 s


In [20]:
titles_count = train.groupby('query_id').label.count()
titles_count = pd.merge(train[['query_id']], titles_count.reset_index(), on='query_id', how='left')
titles_count = titles_count.rename(columns={'label': 'titles_count'})
titles_count2 = test.groupby('query_id').query_title_id.count()
titles_count2 = pd.merge(test[['query_id']], titles_count2.reset_index(), on='query_id', how='left')
titles_count2 = titles_count2.rename(columns={'query_title_id': 'titles_count'})
titles_count = pd.concat([titles_count, titles_count2], ignore_index=True)
# 当前query下的title数量
feat['titles_count'] = titles_count.titles_count
# title中包含的query中的词数
feat['n_include_word'] = extract_feat(n_include_word, query, title)
# 最长子串长度
feat['len_lcsubstr'] = extract_feat(find_lcsubstr, query, title)
# 最长子序列长度
feat['len_lcseque'] = extract_feat(find_lcseque, query, title)
feat['diff_n_include_word'] = feat.len_query - feat.n_include_word
feat['diff_len_lcsubstr'] = feat.len_query - feat.len_lcsubstr
feat['diff_len_lcseque'] = feat.len_query - feat.len_lcseque

time: 27.7 s


In [31]:
# 当前title的序数
feat['query_title_id'] = pd.concat([train['query_title_id'], test['query_title_id']], ignore_index=True)
title_value_counts = title.value_counts().reset_index()
title_value_counts.rename(columns={'index': 'title', 'title': 'title_value_counts'}, inplace=True)
title_value_counts = pd.merge(pd.DataFrame(title), title_value_counts, on='title', how='left')
# title出现次数
feat['title_value_counts'] = title_value_counts['title_value_counts']
#sorensen 相似性系数
feat['sorensen_distance'] = extract_feat(sorensen, query, title)

time: 599 ms


In [33]:
# 载入Word2Vec模型
model = Word2Vec.load("/home/kesci/work/drift_data/drift_word2vec_v100.model")
# 余弦距离
feat['cosine_distance'] = extract_feat(cosine, query, title, sent2vec)
# 曼哈顿距离
feat['cityblock_distance'] = extract_feat(cityblock, query, title, sent2vec)
feat['canberra_distance'] = extract_feat(canberra, query, title, sent2vec)
# 词移距离
feat['wmd'] = extract_feat(model.wv.wmdistance, query, title)
del model
gc.collect()

/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/opt/conda/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


time: 30min 7s


In [37]:
# fuzz特征
feat['fuzz_qratio'] = extract_feat(fuzz.QRatio, query, title)
feat['fuzz_WRatio'] = extract_feat(fuzz.WRatio, query, title)
feat['fuzz_partial_ratio'] = extract_feat(fuzz.partial_ratio, query, title)
feat['fuzz_partial_token_set_ratio'] = extract_feat(fuzz.partial_token_set_ratio, query, title)
feat['fuzz_partial_token_sort_ratio'] = extract_feat(fuzz.partial_token_sort_ratio, query, title)
feat['fuzz_token_set_ratio'] = extract_feat(fuzz.token_set_ratio, query, title)
feat['fuzz_token_sort_ratio'] = extract_feat(fuzz.token_sort_ratio, query, title)

time: 40.8 s


In [65]:
feature_path = "/home/kesci/work/feature_hao.h5"
feature.to_hdf(feature_path, 'feat', mode='w')

time: 14min 24s


In [ ]:
feature_path = "/home/kesci/work/feature_hao.h5"
feat = pd.read_hdf(feature_path, "feat")

In [ ]:
train_sp = 50000000
trainset = lgb.Dataset(feat[:train_sp].values, 
                       label=train.label.values,
                       feature_name=feat.columns.tolist())

In [ ]:
params={'boosting':'gbdt',
        'objective': 'binary',
        'learning_rate': 0.06,
        'max_depth': 6,
        'num_leaves': 31,
        'metric':'auc',
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'subsample_freq': 1,
        'reg_alpha': 5,
        'reg_lambda': 5,
        'n_jobs':16
        }

In [ ]:
# 模型训练
num_rounds = 1000
bst2 = lgb.train(params, trainset, num_rounds)
# 保存模型
bst2.save_model("/home/kesci/work/lgb_final.model")
# 载入模型
bst2 = lgb.Booster(model_file="/home/kesci/work/lgb_final.model")

In [ ]:
del trainset
gc.collect()

In [ ]:
y_pred = bst2.predict(feat[train_sp:])
sub = test[['query_id','query_title_id']]
sub['pred'] = y_pred
sub.to_csv("/home/kesci/work/lgb_hao_final.csv", index=None, header=None)

# 九、lightgbm3 a榜:0.582+

In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [ ]:
# 查看当前kernerl下的package
!pip list --format=columns

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [ ]:
import logging
import sys
from array import array
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from collections import Counter
from gensim.models import word2vec
from keras.preprocessing.text import Tokenizer
import gensim
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from gensim.similarities import WmdSimilarity
from sklearn.metrics import roc_auc_score

def get_log(appname, lv=logging.INFO):
    logger = logging.getLogger(appname)
    formatter = logging.Formatter('%(asctime)s - %(name)s[line:%(lineno)d] - %(levelname)-4s: %(message)s')
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.formatter = formatter
    logger.addHandler(console_handler)
    logger.setLevel(lv)

    return logger

train_path = '/home/kesci/input/bytedance/train_final.csv'
test_path = '/home/kesci/input/bytedance/bytedance_contest.final_2.csv'
train_text_feat_path = '/home/kesci/work/lmm/feature/train_text_feat.csv'
test_text_feat_path = '/home/kesci/work/lmm/feature/test_text_feat.csv'
train_cnt_feat_path = '/home/kesci/work/lmm/feature/train_cnt_feat.csv'
test_cnt_feat_path = '/home/kesci/work/lmm/feature/test_cnt_feat.csv'
train_semantic_feat_path = '/home/kesci/work/lmm/feature/train_semantic_feat.csv'
test_semantic_feat_path = '/home/kesci/work/lmm/feature/test_semantic_feat.csv'
train_w2v_semantic_feat_path = '/home/kesci/work/lmm/feature/train_w2v_semantic_feat.csv'
test_w2v_semantic_feat_path = '/home/kesci/work/lmm/feature/test_w2v_semantic_feat.csv'
prob_path = '/home/kesci/work/lmm/output/add_click.csv'

# 多线程/多进程

In [ ]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from multiprocessing import cpu_count
from functools import wraps
import pandas as pd
import time


class ParallelMachine(object):
    """
        并行机器, 并发执行若干任务

        注: 多进程和多线程会导致内存占用大量增加, 所以避免负载中出现大对象的复制
    """

    def __init__(self):
        self.__func_mapping = []
        self.__result_mapping = {}

    @classmethod
    def do(cls, o):
        """
            多进程要求对象可以序列化, 闭包内的对象无法序列化, 因此存在此方法

            用dill替换pickle

        """
        logger.info(f"using {o[2]}")
        
        start = time.time()
        result = o[0](o[1])
        end = time.time()
        
        return o[2], result  # label, func(data)

    def job(self, mapping=None, handlers=None):
        """
            标记一个函数位需要并发执行,

            :param mapping: 入参和结果集的标签, 示例: {"train": train}
            :param handlers: 同标签数据对应的预处理函数
            :param before: 执行被标记函数前执行的函数
            :param before_args: 前置函数入参
            :param after: 执行被标记函数后执行的函数
            :param after_args: 后置函数入参

            示例
            pm = ParallelMachine()

            @pm.job(mapping={"first_label": first_data, "second_label", second_data})
            def func(data):
                # do something with data
                pass
                # return result
            
        """

        if mapping is None:
            mapping = {}

        def inner(func):
            @wraps(func)
            def wrapper(*args, **kwargs):
                return func(*args, **kwargs)

            for k, v in mapping.items():
                if callable(func):
                    label = func.__name__ + "_" + k
                    self.__func_mapping.append((wrapper, v, label))
                else:
                    raise ValueError(f"[{func}] is not callable")
            return wrapper

        return inner

    def mutilprocess(self, worker=(cpu_count() - 1), timeout=20):
        """
            多进程执行被标记方法们
            :param worker 进程数, 默认是CPU逻辑核心少一个
            :param timeout 每个进程的最大执行时间, 默认20s
    
            注: 多进程必须在main进程下调用
        """

        if __name__ == "__main__":
            with ProcessPoolExecutor(max_workers=worker) as executor:
                self.__result_mapping = {k: v for k, v in executor.map(self.do, self.__func_mapping, timeout=timeout)}
        else:
            raise EnvironmentError("Multiprocess must be under main process")

    @property
    def result(self):
        return self.__result_mapping

In [ ]:
import pandas as pd
import numpy as np

logger = get_log('kesci_Final_')
logger.info('读取训练集数据...')
train = pd.read_csv(train_path, index_col=None, nrows=50000000,names=['query_id','query','query_title_id','title','label'])

In [ ]:
import pandas as pd
import numpy as np

logger.info('读取测试集数据...')
test = pd.read_csv("/home/kesci/input/bytedance/bytedance_contest.final_2.csv", index_col=None, names=['query_id','query','query_title_id','title'])

# 特征工程

### 文本特征（跑完后须要重启kernel运行特征工程前所有cell再提取下一个特征）

In [ ]:
def q_t_loc(data):
    count = []
    for i, row in data.iterrows():
        lens = len(row['query'])
        if row['query'] not in row['title']:
            count.append(-1)
        elif row['query'] in row['title'][:lens]:
            # 判断是否在前
            count.append(0)
        elif row['query'] in row['title'][-lens:]:
            # 判断是否在后
            count.append(2)
        else:
            # 判断是否在中
            count.append(1)
    return count

def q_t_index(data):
    '''
        查看query在title中出现的位置
    '''
    count = []
    for i, row in data.iterrows():
        if row['query'] not in row['title']:
            count.append(-1)
        elif row['query'] in row['title']:
            t = ''.join(row['title'].split(' '))
            q = ''.join(row['query'].split(' '))
            q_index_t = t.index(q)
            count.append(q_index_t)
    return count


def q_t_appear_feat(data):
    '''
        计算query是否完整出现在title中
        计算query出现在title中的位置
    '''
    count = []
    logger.info('计算query是否完整出现在title中...')
    for i, row in data.iterrows():
        if row['query'] in row['title']:
            count.append(1)
        else:
            count.append(0)
    return count

In [ ]:
# 初始化ParallelMachine
# start
pm = ParallelMachine() # 实例化pm对象 

def areas(total, pieces):
    return (((total * i)// pieces, (total * (i + 1)) // pieces) for i in range(pieces))
    
train_mapping = {f"train_{i}": train.iloc[area[0]:area[1], :] for i, area in enumerate(areas(train.shape[0], 14))}
test_mapping = {f"test_{i}": test.iloc[area[0]:area[1], :] for i, area in enumerate(areas(test.shape[0], 14))}

job_mapping = {**train_mapping, **test_mapping}
# end

@pm.job(job_mapping)
def get_text_feat(data):
    '''
        提取文本特征:长度---长度差---index
    '''
    t_f = data[['query_id', 'query','query_title_id', 'title']]
    
    logger.info('提取query长度特征...')
    t_f['query_len'] = [len(i) for i in t_f['query']]
    logger.info('提取title长度特征...')
    t_f['title_len'] = [len(i) for i in t_f['title']]
    logger.info('提取query title长度差特征..')
    t_f['query_title_len_differ'] = t_f['title_len'] - t_f['query_len']
    logger.info('提取query是否在title中特征...')
    t_f['q_t_is_appear'] = q_t_appear_feat(t_f[['query', 'title']])
    
    logger.info('提取query出现在title中的loc...')
    t_f['q_t_loc_appear'] = q_t_loc(t_f[['query', 'title']])
    logger.info('提取query出现在title中的index...')
    t_f['q_t_loc_index'] = q_t_index(t_f[['query', 'title']])
    
    return t_f.drop(['query', 'title'], axis=1)

In [ ]:
# 获取并行执行结果 
# 获取并行执行结果 
from functools import reduce

pm.mutilprocess(worker=12, timeout=None) # 多进程

def merge_parts(resultset, prefix, r):
    return reduce(lambda d1, d2: d1.append(d2), (resultset[f"{prefix}_{i}"] for i in range(r)))

# 文本特征 56min28s
logger.info('提取训练集文本特征...')

train_text_feat = merge_parts(pm.result, "get_text_feat_train", 14) # 执行结果为 函数名_给定标签
# 28min34s

logger.info('提取测试集文本特征...')
test_text_feat = merge_parts(pm.result, "get_text_feat_test", 14) # 执行结果为 函数名_给定标签

# # 2min43ss
logger.info('输出文本特征...')
train_text_feat.to_hdf(train_text_feat_path, '12345', mode='w')
test_text_feat.to_hdf(test_text_feat_path, '12345', mode='w')

### 统计特征（跑完后须要重启kernel运行特征工程前所有cell再提取下一个特征）

In [ ]:
def get_cnt_feat(data):
    '''
        提取统计特征
            query/title word num
            query/title 交集的单词数
            query/title 除去title的单词数/除去query的单词数
            query/title 出现次数
    '''
    c_f = data[['query_id', 'query','query_title_id', 'title']]
    
    logger.info('提取query长度特征...')
    c_f['query_num'] = [len(i.split(' ')) for i in c_f['query']]
    logger.info('提取title长度特征...')
    c_f['title_num'] = [len(i.split(' ')) for i in c_f['title']]
    
    logger.info('提取query/title出现次数...')
    query_cnt = c_f['query'].value_counts().reset_index(name='query_cnt').rename(columns={'index':'query'})
    c_f = c_f.merge(query_cnt, how='left', on='query')
    
    title_cnt = c_f['title'].value_counts().reset_index(name='title_cnt').rename(columns={'index':'title'})
    c_f = c_f.merge(title_cnt, how='left', on='title')
    return c_f.drop(['query', 'title'], axis=1)

In [ ]:
logger.info('提取训练集统计特征...')
train_cnt_feat = get_cnt_feat(train)
train_cnt_feat.to_hdf('/home/kesci/work/lmm/feature/train_cnt_feat.h5', '12345', mode='w')
logger.info('提取测试集统计特征...')
test_cnt_feat = get_cnt_feat(test)
test_cnt_feat.to_hdf('/home/kesci/work/lmm/feature/test_cnt_feat.h5', '12345', mode='w')

### 语义特征（跑完后须要重启kernel运行特征工程前所有cell再提取下一个特征）

In [ ]:
def jaccard(seq1, seq2):
	"""
	The return value is a float between 0 and 1, where 0 means equal, and 1 totally different.
	"""
	set1, set2 = set(seq1), set(seq2)
	return 1 - len(set1 & set2) / float(len(set1 | set2))


def sorensen(seq1, seq2):
	"""
	The return value is a float between 0 and 1, where 0 means equal, and 1 totally different.
	"""
	set1, set2 = set(seq1), set(seq2)
	return 1 - (2 * len(set1 & set2) / float(len(set1) + len(set2)))


def levenshtein(seq1, seq2, max_dist=-1):
    """
    seq1, seq2:空格分隔的句子
    """
    if seq1 == seq2:
        return 0
    seq1, seq2 = seq1.split(), seq2.split()
    len1, len2 = len(seq1), len(seq2)
    if max_dist >= 0 and abs(len1 - len2) > max_dist:
        return -1
    if len1 == 0:
        return len2
    if len2 == 0:
        return len1
    if len1 < len2:
        len1, len2 = len2, len1
        seq1, seq2 = seq2, seq1
    
    column = array('L', range(len2 + 1))
    
    for x in range(1, len1 + 1):
        column[0] = x
        last = x - 1
        for y in range(1, len2 + 1):
            old = column[y]
            cost = int(seq1[x - 1] != seq2[y - 1])
            column[y] = min(column[y] + 1, column[y - 1] + 1, last + cost)
            last = old
        if max_dist >= 0 and min(column) > max_dist:
            return -1
    
    if max_dist >= 0 and column[len2] > max_dist:
        # stay consistent, even if we have the exact distance
        return -1
    return column[len2]


def cos_dist(w1, w2):
    dist1=float(np.dot(w1, w2)/(np.linalg.norm(w1)*np.linalg.norm(w2)))
    return dist1


def get_min_max_scaler(x):
    return pd.Series((x-x.min()) / (x.max() - x.min()))


def cos_feat(query, title):
    cos_q_t = []
    count = 0
    wv1 = []
    wv2 = []
    for q, t in zip(query, title):
        count += 1
        key_word = list(set(q + t))
        word_vector1 = np.zeros(len(key_word))
        word_vector2 = np.zeros(len(key_word))
        
        for i in range(len(key_word)):
            for j in range(len(q)):
                if key_word[i] == q[j]:
                    word_vector1[i] += 1
            wv1.append(word_vector1)
            for k in range(len(t)):
                if key_word[i] == t[k]:
                    word_vector2[i] += 1
            wv2.append(word_vector2)
            
        cos_q_t.append(cos_dist(word_vector1, word_vector2))
    
    return cos_q_t

# start
pm = ParallelMachine() # 实例化pm对象 

def areas(total, pieces):
    return (((total * i)// pieces, (total * (i + 1)) // pieces) for i in range(pieces))
    
train_mapping = {f"train_{i}": train.iloc[area[0]:area[1], :] for i, area in enumerate(areas(train.shape[0], 14))}
test_mapping = {f"test_{i}": test.iloc[area[0]:area[1], :] for i, area in enumerate(areas(test.shape[0], 14))}

job_mapping = {**train_mapping, **test_mapping}
# end

@pm.job(job_mapping)
def get_semantic_feat(data):
    '''
        相似度
    '''
    s_f = data[['query_id', 'query', 'query_title_id', 'title']]
    logger.info('get jaccard distance...')
    s_f['q_t_jaccard'] = [jaccard(i, j) for i, j in zip(s_f['query'], s_f['title'])]
    
    logger.info('get sorensen distance...')
    s_f['q_t_sorensen'] = [sorensen(i, j) for i, j in zip(s_f['query'], s_f['title'])]
    
    logger.info('get levenshtein distance...')
    s_f['q_t_levenshtein'] = [levenshtein(i, j) for i, j in zip(s_f['query'], s_f['title'])]

    logger.info('计算余弦相似度...')    
    s_f['cos_q_t'] = cos_feat(data['query'], data['title'])

    return s_f.drop(['query', 'title'], axis=1)

In [ ]:
from functools import reduce

pm.mutilprocess(worker=10, timeout=None) # 多进程
def merge_parts(resultset, prefix, r):
    return reduce(lambda d1, d2: d1.append(d2), (resultset[f"{prefix}_{i}"] for i in range(r)))
    
logger.info('提取训练集语义...') 
train_semantic_feat = merge_parts(pm.result, "get_semantic_feat_train", 14) # 执行结果为 函数名_给定标签 

# 19min58s
logger.info('提取测试集语义...') 
test_semantic_feat = merge_parts(pm.result, "get_semantic_feat_test", 14) # 执行结果为 函数名_给定标签

train_semantic_feat.to_hdf('/home/kesci/work/lmm/feature/train_semantic_feat.h5', '12345', mode='w')
test_semantic_feat.to_hdf('/home/kesci/work/lmm/feature/test_semantic_feat.h5', '12345', mode='w')

### 交集特征（跑完后须要重启kernel运行特征工程前所有cell再提取下一个特征）

In [ ]:
# start
pm = ParallelMachine() # 实例化pm对象 

def areas(total, pieces):
    return (((total * i)// pieces, (total * (i + 1)) // pieces) for i in range(pieces))
    
train_mapping = {f"train_{i}": train.iloc[area[0]:area[1], :] for i, area in enumerate(areas(train.shape[0], 14))}
test_mapping = {f"test_{i}": test.iloc[area[0]:area[1], :] for i, area in enumerate(areas(test.shape[0], 14))}

job_mapping = {**train_mapping, **test_mapping}
# end

@pm.job(job_mapping)
def intersection_words_feat(data):
    sum_q_t_intersection = []
    mean_q_t_intersection = []
    var_q_t_intersection = []
    for i, j in zip(data['query'], data['title']):
        q_t = []
        q = i.split(' ')
        t = j.split(' ')
        for k in q:
            q_t.append(t.count(k))
        sum_q_t_intersection.append(sum(q_t))
        mean_q_t_intersection.append(sum(q_t)/len(q_t))
        var_q_t_intersection.append(np.array(q_t).var())
    i_w_f = data[['query_id', 'query_title_id']]
    i_w_f['q_t_intersection_sum_cnt'] = sum_q_t_intersection
    i_w_f['q_t_intersection_mean_cnt'] = mean_q_t_intersection
    i_w_f['q_t_intersection_var_cnt'] = var_q_t_intersection
    return i_w_f

In [ ]:
from functools import reduce

pm.mutilprocess(worker=10, timeout=None) # 多进程
def merge_parts(resultset, prefix, r):
    return reduce(lambda d1, d2: d1.append(d2), (resultset[f"{prefix}_{i}"] for i in range(r)))
logger.info('交集出现次数')
train_q_t_intersection = merge_parts(pm.result, "intersection_words_feat_train", 14) # 执行结果为 函数名_给定标签

logger.info('交集出现次数')
test_q_t_intersection = merge_parts(pm.result, "intersection_words_feat_test", 14) # 执行结果为 函数名_给定标签

train_q_t_intersection.to_hdf('/home/kesci/work/lmm/feature/train_q_t_intersection.h5', '12345', mode='w')
test_q_t_intersection.to_hdf('/home/kesci/work/lmm/feature/test_q_t_intersection.h5', '12345', mode='w')

# 模型

In [ ]:
def lgb_predict(train_f, train_label, val_f, val_label):
    lgb_train = lgb.Dataset(train_f, train_label)
    lgb_eval = lgb.Dataset(val_f, val_label)
    # specify your configurations as a dict

    params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'boost_from_average': False,
        'metric': 'auc',
        'num_leaves': 30,
        'min_data_in_leaf':50,
        'max_depth':5,
        'learning_rate': 0.01,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'n_jobs':4
    }
    print('Start training...')
    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=1000,
                    valid_sets=lgb_eval)
    print('Save model...')
    # save model to file
    gbm.save_model('/home/kesci/work/lmm/model/lgb_model.txt')

### 重启kernel运行特征工程前所有cell再运行以下cell

In [ ]:
train_text_feat = pd.read_hdf('/home/kesci/work/lmm/feature/train_text_feat.csv', '12345')
train_semantic_feat = pd.read_hdf('/home/kesci/work/lmm/feature/train_semantic_feat.h5', '12345')
train_cnt_feat = pd.read_hdf('/home/kesci/work/lmm/feature/train_cnt_feat.h5', '12345')
train_q_t_intersection = pd.read_hdf('/home/kesci/work/lmm/feature/train_q_t_intersection.h5', '12345')

In [ ]:
logger.info('merge 文本特征...')
train_feat = train[['query_id', 'query_title_id', 'label']].merge(train_text_feat, on=['query_id', 'query_title_id'], how='left')
logger.info('merge 统计特征...')
train_feat = train_feat.merge(train_cnt_feat, on=['query_id', 'query_title_id'], how='left')
logger.info('merge 语义特征...')
train_feat = train_feat.merge(train_semantic_feat, on=['query_id', 'query_title_id'], how='left')
logger.info('merge 交集单词出现次数特征...')
train_feat = train_feat.merge(train_q_t_intersection, on=['query_id', 'query_title_id'], how='left')

In [ ]:
del train_text_feat
del train_semantic_feat
del train_cnt_feat
del train_q_t_intersection
import gc
gc.collect()

In [ ]:
train_f = train_feat[-48000000:].drop(['query_id', 'query_title_id', 'label'], axis=1)
val_f = train_feat[:2000000].drop(['query_id', 'query_title_id', 'label'], axis=1)

train_label = train_feat['label'][-48000000:]
val_label = train_feat['label'][:2000000]

lgb_predict(train_f, train_label, val_f, val_label, test_f)

### 重启kernel运行特征工程前所有cell再运行以下cell

In [ ]:
test_text_feat = pd.read_hdf('/home/kesci/work/lmm/feature/test_text_feat.csv', '12345')
test_semantic_feat = pd.read_hdf('/home/kesci/work/lmm/feature/test_semantic_feat.h5', '12345')
test_cnt_feat = pd.read_hdf('/home/kesci/work/lmm/feature/test_cnt_feat.h5', '12345')
test_q_t_intersection = pd.read_hdf('/home/kesci/work/lmm/feature/test_q_t_intersection.h5', '12345')

logger.info('merge 文本特征...')
test_feat = test[['query_id', 'query_title_id']].merge(test_text_feat, on=['query_id', 'query_title_id'], how='left')
logger.info('merge 统计特征...')
test_feat = test_feat.merge(test_cnt_feat, on=['query_id', 'query_title_id'], how='left')
logger.info('merge 语义特征...')
test_feat = test_feat.merge(test_semantic_feat, on=['query_id', 'query_title_id'], how='left')
logger.info('merge 交集单词出现次数特征...')
test_feat = test_feat.merge(test_q_t_intersection, on=['query_id', 'query_title_id'], how='left')

test_f = test_feat.drop(['query_id', 'query_title_id'], axis=1)

In [ ]:
del test_text_feat
del test_semantic_feat
del test_cnt_feat
del test_q_t_intersection
import gc
gc.collect()

In [ ]:
gbm = lgb.Booster(model_file='/home/kesci/work/lmm/model/lgb_model.txt')
pred = gbm.predict(test_f, num_iteration=gbm.best_iteration)
feat_importances = pd.DataFrame({
    'column': test_f.columns,
    'importance': gbm.feature_importance(),
}).sort_values(by='importance')
plt.figure(figsize=(12,6))
lgb.plot_importance(gbm, max_num_features=30)
plt.title("Featurertances")
plt.show()
pre = list(pred)
prob = test[['query_id', 'query_title_id']]
prob['label'] = pre
prob.to_csv('/home/kesci/work/lmm/output/final_test.csv', index=False, header=None)

# 十、融合方案(线性加权)，最终成绩a榜：0.619+，b榜：0.645+

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [ ]:
# 最终预测
r1 = pd.read_csv("stage2_improved_textcnn_feat_submit_v0.csv", names=['query_id','query_title_id', 'pred'])
r2 = pd.read_csv("stage2_LSTM_FastText_feat_submit_Epoch1.csv", names=['query_id','query_title_id', 'pred'])
r3 = pd.read_csv("lgb_hao_final.csv", names=['query_id','query_title_id', 'pred'])
r4 = pd.read_csv("stage2_lgb_submit_v0.csv", names=['query_id','query_title_id', 'pred'])
r5 = pd.read_csv("lmm/output/final_test.csv", names=['query_id','query_title_id', 'pred'])
new = pd.read_csv("stage2_LSTM_CNN_feat_submit_Epoch3.csv", names=['query_id','query_title_id', 'pred'])

In [ ]:
# 两个603 nn
res = r1.copy()
res.pred = r1.pred * 0.5 + r2.pred * 0.5
# 三个lgb
res2 = r1.copy()
res2.pred = r3.pred * 0.4 + r4.pred * 0.4 + + r5.pred * 0.2
# nn和lgb
fuse1 = r1.copy()
fuse1.pred = res.pred * 0.6 + res2.pred * 0.4
# 融610 nn
fuse5 = fuse1.copy()
fuse5.pred = fuse1.pred * 0.6 + new.pred * 0.4
fuse5.pred = fuse5.pred.astype('float32')

In [ ]:
fuse5.to_csv("/home/kesci/work/final1.csv", index=None, header=None)

In [ ]:
!https_proxy="http://klab-external-proxy" ./kesci_submit -file final1.csv -token 385da1c2ad16422b -mode archive